# SEL634 - Laboratório de Robôs Manipuladores

## Inicialização da biblioteca

Carregar a biblioteca de robótica para Python:

In [ ]:
!python3 -m pip install --user --upgrade pip==20.1
!pip install roboticstoolbox-python
!pip install spatialmath-python

In [ ]:
import numpy as np
#import roboticstoolbox as rtb
from roboticstoolbox import *
from spatialmath import *
from math import pi
import matplotlib.pyplot as plt
from matplotlib import cm
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

# no Binder utilizar essa linha
%matplotlib notebook  
# no Colab utilizar essa linha
# %matplotlib inline

## Cinemática Direta do Robô Kinova Gen3

O robô Kinova Gen3 pode ser modelado pela seguinte função (ver Tarefa 2):

In [ ]:
def kinova_robot(my_name):
  robot = DHRobot(
      [
       # Colocar seu código aqui para completar a descrição do robô
       RevoluteDH(alpha=pi,d=-(0.1059+0.0615),offset=pi) # Linha para a junta 7
       ], name=my_name)
  return robot


A pose do efetuador do robô Kinova Gen3 é dada em relação a base, $B$, pela função:

In [ ]:
def pose_efetuador(robot, q):
  H_B0 = SE3.Rz(0)*SE3(0,0,0)*SE3(0,0,0)*SE3.Rx(pi)
  H_07 = robot.fkine(q)
  return H_B0*H_07

A biblioteca de robótica permite acessar as **matrizes de transformação homogênea intermediárias do modelo de cinemática direta do robô**. 

Por exemplo, seja a variável `robot` o modelo retornado pela função `kinova_robot` criada anteriormente. O comando:

```
H = robot.A([j1,j2],q)
```

retorna a matriz de transformação homogênea `H` da junta `j2` em relação a junta `j1` para a configuração do robô `q`. 

Portanto, para obter a matriz de transformação homogênea $^0H_1$ para uma configuração `q` fornecida:
```
H = robot.A([0,1],q)
```

A biblioteca de robótica possui uma função que retorna o **jacobiano do manipulador** para uma configuração `q`:
```
J = robot.jacobe(q)
```
O comando acima retorna o Jacobiano completo $6 \times N$. 

Para obter apenas a parte superior do Jacobiano, que está relacionado a velocidade linear:
```
J = robot.jacobe(q,half='trans')
```
Para obter apenas a parte inferior do Jacobiano, que está relacionado a velocidade angular:
```
J = robot.jacobe(q,half='rot')
```

In [ ]:
robot = kinova_robot("meu_robo")
q = np.array([0,0,0,0,0,0,0])
robot.A([0,1],q)

## Trajetória do Robô no Espaço das Juntas

Considere que a junta $i$ possui a seguinte trajetória  em graus no tempo que leva a junta $i$ da posição $\theta_{si}$ até $\theta_{gi}$ em um dado tempo $T_d$:
$$ \theta_i (t) = c_{0i} + c_{1i} t + c_{2i} t^2 + c_{3i} t^3 + c_{4i} t^4 + c_{5i} t^5 $$
Para velocidades e acelerações iniciais e finais iguais a zero, os coeficientes do polinômio são dados em função da posição inicial e final e do intervalo de tempo:
\begin{eqnarray*}
	&c_{0i}& = \theta_{si}\\
	&c_{1i}& = c_{2i} = 0\\
	&c_{3i}& = \frac{10(\theta_{gi}-\theta_{si})}{T_d^{3}}\\
	&c_{4i}& = \frac{15(\theta_{si}-\theta_{gi})}{T_d^{4}}\\
	&c_{5i}& = \frac{6(\theta_{gi}-\theta_{si})}{T_d^{5}}
\end{eqnarray*}

A velocidade de cada junta para condições iniciais nulas é dada por:
$$ \dot\theta_i (t) = c_{1i} + 2c_{2i} t + 3c_{3i} t^2 + 4c_{4i} t^3 + 5c_{5i} t^4 $$

A aceleração de cada junta para condições iniciais nulas é dada por:
$$ \ddot\theta_i (t) = 2c_{2i} + 6c_{3i} t + 12c_{4i} t^2 + 20c_{5i} t^3 $$

A função a seguir foi implementada na Tarefa 3 para calcular a trajetória das juntas do manipulador:


In [ ]:
def calcular_trajetoria(theta_s, theta_g, Td, dt):
  # Coloque seu codigo aqui

  return theta_t, dtheta_t, ddtheta_t

A função a seguir foi implementada na Tarefa 3 para calcular a sequencia no tempo de matrizes de transformação homogênea que representa a pose do robô (posição e orientação do efetuador)

In [ ]:
def calcular_traj_efetuador(robot, theta_t):
  # Coloque seu codigo aqui

return H_efetuador

## 4. Velocidade do Efetuador do Robô Manipulador

Na tarefa 4, você vai implementar funções para calcular a velocidade do efetuador do manipulador Kinova Gen3 em relação a sua base $B$.

### 4.1 Jacobiano do Manipulador

Implemente uma função semelhante àquela disponível na biblioteca de robótica para calcular o Jacobiano do manipulador em relação ao sistema de coordenada $\{0\}$. Nesta etapa não considere a transformação homogênea $^BH_0$. 

**Entrada:** modelo cinemático do robô, e o vetor de configuração do robô em um determinado instante de tempo ($6 \times 1$).

**Saída:** o Jacobinado do efetuador em relação ao sistema de coordenadas $\{0\}$ para um instante de tempo, ou seja, matriz $6 \times N$.


In [ ]:
def calcular_jacobiano(robot,theta):
  # Coloque seu codigo aqui

  return J_efetuador

### 4.2 Velocidade do Manipulador

Implemente uma função que calcula a velocidade do efetuador do Kinova Gen3 em relação a base. Para isso utilize a função implementada no item 4.1.

O calculo das velocidades linear e angular do manipulador em relação ao sistema $\{0\}$ para um dado instante de tempo é:

$$\begin{bmatrix} ^0v_n(t) \\ ^0w_n(t) \end{bmatrix} = {^0J_n}(q(t)) ~\dot q(t) $$

Como a origem do sistema de coordenadas $\{0\}$ e $\{B\}$ coincidem, para transformar o vetor de velocidades para o sistema de coordenadas da base $\{B\}$:

$$ \begin{bmatrix} ^Bv_n(t) \\ ^Bw_n(t) \end{bmatrix} = {^BR_0}\begin{bmatrix} ^Bv_n(t) \\ ^Bw_n(t) \end{bmatrix} $$

**Entrada:** modelo cinemático do robô, vetor de configuração do robô no tempo ($N \times M$), velocidade das juntas do robô no tempo ($N \times M$), onde $M$ é a quantidade de pontos da trajetória no tempo (ver Tarefa 3).

**Saída:** Velocidade linear (m/s) e angular (rad/s) do efetuador em relação ao sistema de coordenadas $\{B\}$ ao longo do tempo (matriz $6 \times M$) 

In [ ]:
def calcular_vel_efetuador(robot,theta_t,dtheta_t):
  # Coloque seu codigo aqui


  return V_efetuador

### 4.3 Velocidade do Manipulador - Calculo alternativo

Implemente a mesma função anterior, mas utilizando o calculo de Jacobiano feito pela biblioteca de robótica através da função: 

`J = robot.jacobe(q)`



In [ ]:
def calcular_vel_efetuador_alternativo(robot,theta_t,dtheta_t):
  # Coloque seu codigo aqui


  return V_efetuador

### 4.4 Plotar velocidade do efetuador

Implemente uma função para gerar dois gráficos: um gráfico para mostrar a evolução no tempo de cada componente da velocidade linear; outro gráfico para mostrar no tempo a evoluçào da velocidade angular. Utilize as funções implementadas no item 4.1 e 4.2.


In [ ]:
def plotar_vel_efetuador(robot, theta_t, dtheta_t, Td, dt):
  # Coloque seu codigo aqui

Faça outra função para gerar os mesmos gráficos de velocidade, mas utilizando a função implementada no item 4.3.

In [ ]:
def plotar_vel_efetuador_alternativo(robot, theta_t, dtheta_t, Td, dt):
  # Coloque seu codigo aqui

Os gráficos gerados devem ser iguais. 

### 4.5 Teste das funções

Faça o teste das funções do item 4.4 para os seguintes parâmetros: $T_d = 10$ segundos, $\delta t = 0.01$ segundos,  posição inicial das juntas $\theta_s = [0, 0, 0, 0, 0, 0, 0]^T$  e posição final das juntas $\theta_g = [90, 90, 0, -90, 0, 90, 0]^T$ dada em graus.

In [ ]:
Td = 10.0
dt = 0.01
theta_s = np.array([0,0,0,0,0,0,0])
theta_g = np.array([90,90,0,−90,0,90,0])

